# Datenmanipulation mit Pandas in Python

## DataFrames einlesen: CSV, Excel, URL und wichtige Parameter

Pandas kann fast jedes Datenformat einlesen. Die wichtigsten Funktionen sind:

| Funktion           | Typische Quelle                         |
|--------------------|-----------------------------------------|
| `pd.read_csv()`    | CSV, TSV, Textdateien                   |
| `pd.read_excel()`  | .xlsx, .xls                             |
| `pd.read_json()`   | JSON, APIs                              |
| `pd.read_html()`   | Tabellen aus Webseiten                  |
| direkt aus URL     | beinhaltet obige Funktionen, ohne es vorher herunter zu laden |

### CSV einlesen: `pd.read_csv()` mit den wichtigsten Parametern

Die wichtigsten Parameter der Funktion sind Folgende:

| Parameter         | Wirkung                                      | Typisches Problem ohne diesen Parameter      |
|-------------------|----------------------------------------------|-----------------------------------------------|
| `sep` oder `delimiter` | Trenner in der CSV (oft `;`, `|`, `\t`)  | Spalten werden falsch getrennt                |
| `encoding`        | Zeichenkodierung (`utf-8`, `latin1`, `cp1252`) | Umlaute oder Fehler                       |
| `decimal`         | Dezimaltrennzeichen (`,` statt `.`)          | 52.000,50 wird zu String                      |
| `thousands`       | Tausendertrennzeichen (`.` oder Leerzeichen) | Zahlen werden als String gelesen              |
| `parse_dates`     | Spalten automatisch als Datum parsen         | Datum bleibt String                           |
| `dtype`           | Datentypen explizit setzen                   | Pandas rät falsch (z.B. ID als float)         |
| `usecols`         | Nur bestimmte Spalten laden                  | Speicher und Zeit sparen                        |
| `nrows`           | Nur erste n Zeilen laden                     | Schnell reinschauen                           |


#### Beispiel: Deutsche CSV mit Semikolon, Komma als Dezimal und Umlauten einlesen

Bevor man mit echten Dateien arbeitet, erstelle ich ein kleines, aber realistisches Beispiel:

```python
import pandas as pd
import numpy as np

data = {
    'Name': ['Anna', 'Ben', 'Clara', 'David', 'Emma'],
    'Alter': [28, 34, 19, 42, 31],
    'Gehalt_EUR': ['52.000,50', '68.000,00', '45.000,75', '82.000,00', '59.000,25'],
    'Eintritt': ['2021-03-15', '2020-11-01', '2023-07-20', '2019-01-10', '2022-05-05'],
    'Abteilung': ['IT', 'Vertrieb', 'IT', 'Vertrieb', 'Marketing']
}
df_demo = pd.DataFrame(data)
df_demo.to_csv('mitarbeiter_demo.csv', index=False, sep=';', decimal=',')
df_demo.to_excel('mitarbeiter_demo.xlsx', index=False)

print("Demo-Dateien erstellt: mitarbeiter_demo.csv und mitarbeiter_demo.xlsx")
```

Die Datei enthält typisch deutsche „Problemfälle“:
- Gehalt mit Punkt als Tausender- und Komma als Dezimaltrennzeichen ( 52.000,50)
- Semikolon als Spaltentrenner
- Datum im Format YYYY-MM-DD
- Umlaute und Sonderzeichen

So kann man alle wichtigen Parameter von `pd.read_csv()` und `pd.read_excel()` direkt testen, ohne dass man erst eine externe Datei herunterladen muss.

```python
df = pd.read_csv(
    'mitarbeiter_demo.csv',
    sep=';',                    # Deutscher Standard: Semikolon
    encoding='utf-8',           # oder 'latin1' bei alten Excel-Exports
    decimal=',',                # 52.000,50: float 52000.50
    thousands='.',              # 52.000,50: korrekt erkannt
    parse_dates=['Eintritt'],   # Spalte als datetime
    dtype={'Name': 'string', 'Abteilung': 'category'},  # Speicher sparen
    usecols=['Name', 'Alter', 'Gehalt_EUR', 'Eintritt', 'Abteilung'],  # nur diese Spalten
    nrows=None                   # None = alle Zeilen (oder z.B. 100 zum Testen)
)

print("Erfolgreich eingelesen!")
df.info()
df.head()
```

### Excel-Dateien einlesen: `pd.read_excel()`

Mit `pd.read_excel()` kann man .xlsx- und .xls-Dateien direkt einlesen, ohne Excel öffnen zu müssen. Wichtige Parameter sind sheet_name (welches Tabellenblatt?), skiprows (Überschriften oder Metadaten überspringen), usecols (nur bestimmte Spalten oder Bereiche wie „A:F“ laden) und dtype, um z. B. Kundennummern als String statt als Zahl zu erzwingen. Pandas kann Excel-Dateien sogar direkt aus einer URL laden. 

```python
df_excel = pd.read_excel(
    'mitarbeiter_demo.xlsx',
    sheet_name='Sheet1',        # oder 0 für erstes Blatt
    usecols='A:E',              # Excel-Spaltenbereich
    skiprows=2,                 # z.B. Überschriften überspringen
    dtype={'Gehalt_EUR': str}  # falls Excel komische Formate hat
)
```

### URL einlesen: 

(Demnächst)

## Selektion & Indexing in Pandas

### Selektion mittels `.loc[]`, `.iloc[]` und `df[]`

Die Bibliothek Pandas bietet mehrere Wege, um Zeilen und Spalten eines DataFrames auszuwählen. Je nach Situation ist eine Methode besser als die Andere.

| Methode         | Was sie macht                         | Empfohlen für                     |
|-----------------|---------------------------------------|-----------------------------------|
| `df[]`          | Einfache Auswahl (Spalten oder bool)  | Schnell und einfach                 |
| `.loc[]`        | Labelbasiert (Namen der Spalten/Zeilen)| Immer dann, wenn man Namen verwendet  |
| `.iloc[]`       | Positionsbasiert (Indexnummer der Spalten/Zeilen)    | Wenn man mit Position arbeitet und es größere Dataframes sind   |


`.loc[]` und `.iloc[]` sind sicherer als `[]`, weil sie keine Verwirrung zwischen Label und Position zulassen. Die einfachen eckigen Klammern `df[]` sind zwar praktisch, können aber zu unerwartetem Verhalten führen, wenn sich Spaltennamen und Indexwerte überschneiden. `.loc[]` und `.iloc[]` hingegen sind explizit:  
- **`.loc[]`** arbeitet ausschließlich mit Spalten- und Zeilennamen,  
- **`.iloc[]`** ausschließlich mit numerischen Positionen (Indexen).  
  
Das Wichtigste bei `.loc[]` ist, dass das Ende des Slices immer inklusiv ist.

```python
df.loc['2020-01-01':'2020-12-31']    # Hier wird der 31. Dezember 2020 mit ausgegeben
df.loc['P2':'P5']                    # P2, P3, P4 und P5 werden angezeigt
df.loc[10:50]                        # wenn der Index aus Zahlen besteht, so sind 10 bis 50 inklusive
```

Die `.iloc[]` Funktion ist positionsbasiert und es zählt nur die numerische Position. Wichtig zu bemerken ist, genau wie bei Listen gilt, dass das Ende des Slices exklusiv (nicht mit eingeschlossen) ist.

```python
df.iloc[0:5]     # Zeilen an Position 0, 1, 2, 3, 4. Die 5. Zeile (Index 4) ist die letzte
df.iloc[1:4]     # Position 1, 2, 3
df.iloc[-3:]     # die letzten drei Zeilen 
```
 
#### Direkter Vergleich zwischen `.loc[]` und `.iloc[]`:

Angenommen, unser DataFrame hat den Index `['A', 'B', 'C', 'D', 'E']`:

```python
df.loc['B':'D']   # Diese Methode liefert B, C und D (D ist inklusiv)
df.iloc[1:4]      # liefert ebenfalls B, C, D (D ist exklusiv)

df.loc['B':'E']   # liefert B, C, D und E (E ist dabei)
df.iloc[1:5]      # liefert B, C, D und E (erst bei Position 5 wird E erreicht)
df.iloc[1:4]      # würde nur B, C, D liefern (E fehlt)
```

### Boolsche Masken und komplexe Bedingungen

Eine weitere mächtigste Funktionen von Pandas ist die Filterung mit booleschen Masken. Statt Schleifen zu schreiben, erzeugt man einen booleschen Vektor (True/False) und übergibt ihn an das DataFrame:

```python
df[df['Alter'] > 30]                                   # einfache Bedingung
df[(df['Stadt'] == 'Berlin') & (df['Gehalt'] > 60000)] # Konjunktion
df[(df['Abteilung'] == 'IT') | (df['Alter'] < 25)]     # Disjunktion
```

Wichtig zu bemerken ist, dass bei mehreren Bedingungen runde Klammern um jede einzelne Bedingung stehen müssen und die Operatoren `&` (und), `|` (oder) sowie `~` (nicht) werden verwendet.


## Neue Spalten erstellen & löschen

In Pandas kann man DataFrames dynamisch erweitern oder Spalten entfernen. Ich starte mit einfachen Berechnungen und Zuweisungen, bevor ich zu fortgeschrittenen Methoden kommen.

### Einfache Berechnungen und Zuweisungen

Die einfachste Methode, eine neue Spalte zu erstellen, ist die direkte Zuweisung mit `df['neue_spalte'] = ...`. Man kann arithmetische Operationen, Konstanten oder bestehende Spalten kombinieren. Pandas wendet die Operation vektoriell aus.
Dazu lade ich wieder eine Beispieldatei:

In [4]:
import pandas as pd
import numpy as np

data = {
    'Name': ['Anna', 'Ben', 'Clara', 'David', 'Emma'],
    'Alter': [28, 34, 19, 42, 31],
    'Gehalt_EUR': [52000.50, 68000.00, 45000.75, 82000.00, 59000.25],
    'Eintritt': pd.to_datetime(['2021-03-15', '2020-11-01', '2023-07-20', '2019-01-10', '2022-05-05']),
    'Abteilung': pd.Categorical(['IT', 'Vertrieb', 'IT', 'Vertrieb', 'Marketing'])
}
df = pd.DataFrame(data)

print("Original-DataFrame:")
print(df.head())

# Einfache Zuweisung: Konstante Spalte hinzufügen
df['Land'] = 'Deutschland'  # Jede Zeile bekommt denselben Wert

# Arithmetische Berechnung: Neue Spalte "Gehalt_pro_Alter" = Gehalt / Alter
df['Gehalt_pro_Alter'] = df['Gehalt_EUR'] / df['Alter']

# String-Verkettung: Vollständiger Name (angenommen, wir haben eine 'Nachname'-Spalte – hier simuliert)
df['Nachname'] = ['Müller', 'Schmidt', 'Fischer', 'Weber', 'Meyer']  # Beispiel
df['Vollname'] = df['Name'] + ' ' + df['Nachname']

print("\nDataFrame nach neuen Spalten:")
print(df[['Name', 'Gehalt_EUR', 'Alter', 'Gehalt_pro_Alter', 'Vollname', 'Land']].head())

Original-DataFrame:
    Name  Alter  Gehalt_EUR   Eintritt  Abteilung
0   Anna     28    52000.50 2021-03-15         IT
1    Ben     34    68000.00 2020-11-01   Vertrieb
2  Clara     19    45000.75 2023-07-20         IT
3  David     42    82000.00 2019-01-10   Vertrieb
4   Emma     31    59000.25 2022-05-05  Marketing

DataFrame nach neuen Spalten:
    Name  Gehalt_EUR  Alter  Gehalt_pro_Alter       Vollname         Land
0   Anna    52000.50     28       1857.160714    Anna Müller  Deutschland
1    Ben    68000.00     34       2000.000000    Ben Schmidt  Deutschland
2  Clara    45000.75     19       2368.460526  Clara Fischer  Deutschland
3  David    82000.00     42       1952.380952    David Weber  Deutschland
4   Emma    59000.25     31       1903.233871     Emma Meyer  Deutschland


- apply() und lambda-Funktionen
- map() und replace()
- Bedingte Spalten mit np.where() und np.select()
- drop(), pop(), del


## Daten bereinigen (Cleaning)
### Umgang mit fehlenden Werten (isnull, dropna, fillna)

Fehlende Werte (NaN) können Analysen verzerren, z.B. bei der Berechnung von Renditen. Man kann sie mit isnull() oder isna() überprüfen, entfernen mit dropna() oder füllen mit fillna().
Zuerst erstelle ich ein DataFrame mit NaNs in der 'Preis'-Spalte.




In [5]:
import pandas as pd
import numpy as np

data = {
    'Datum': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'Aktie': ['AAPL', 'GOOG', 'TSLA', 'MSFT', 'AMZN'],
    'Preis': [150.5, np.nan, 250.0, np.nan, 100.2],
    'Volumen': [1000, 1500, 2000, 1200, 800],
    'Kategorie': ['Tech', 'Tech', 'EV', 'Tech', 'Retail']
}
df = pd.DataFrame(data)
df

,Datum,Aktie,Preis,Volumen,Kategorie
0,2023-01-01,AAPL,150.5,1000,Tech
1,2023-01-02,GOOG,NaN,1500,Tech
2,2023-01-03,TSLA,250.0,2000,EV
3,2023-01-04,MSFT,NaN,1200,Tech
4,2023-01-05,AMZN,100.2,800,Retail


In [6]:
# Fehlende Werte identifizieren
print(df.isnull().sum())  # Pro Spalte
print("\nDataFrame mit NaN-Markierungen:")
df.isnull()

Datum        0
Aktie        0
Preis        2
Volumen      0
Kategorie    0
dtype: int64

DataFrame mit NaN-Markierungen:


,Datum,Aktie,Preis,Volumen,Kategorie
0,False,False,False,False,False
1,False,False,True,False,False
2,False,False,False,False,False
3,False,False,True,False,False
4,False,False,False,False,False


In [7]:
# Zeilen mit NaNs entfernen
df_dropna = df.dropna(subset=['Preis'])  # Nur in 'Preis'-Spalte
df_dropna

,Datum,Aktie,Preis,Volumen,Kategorie
0,2023-01-01,AAPL,150.5,1000,Tech
2,2023-01-03,TSLA,250.0,2000,EV
4,2023-01-05,AMZN,100.2,800,Retail


In [8]:
# NaNs mit Mittelwert füllen
df_fillna = df.copy()
df_fillna['Preis'] = df_fillna['Preis'].fillna(df['Preis'].mean())
df_fillna

,Datum,Aktie,Preis,Volumen,Kategorie
0,2023-01-01,AAPL,150.5,1000,Tech
1,2023-01-02,GOOG,166.9,1500,Tech
2,2023-01-03,TSLA,250.0,2000,EV
3,2023-01-04,MSFT,166.9,1200,Tech
4,2023-01-05,AMZN,100.2,800,Retail



- Duplikate finden und entfernen
- Datentypen konvertieren (astype, pd.to_datetime, pd.to_numeric)
- String-Methoden (.str.) – lower, upper, contains, replace, split, strip


## Sortieren, Ranking & Reihenfolge

Pandas bietet zwei Hauptmethoden zum Sortieren:

`sort_values()`: Sortiert nach Werten in Spalten.
`sort_index()`: Sortiert nach dem Index.

Diese Methoden sind inplacefähig und erlauben aufsteigende/absteigende Reihenfolge (ascending=True/False).
Zuerst importieren wir Pandas und erstellen ein Beispiel-DataFrame mit Aktien-Daten (Rendite, Volatilität, Sharpe-Ratio).


In [9]:
import pandas as pd
import numpy as np

# Beispiel-DataFrame: Fiktive Aktien-Daten
data = {
    'Aktie': ['AAPL', 'GOOG', 'TSLA', 'MSFT', 'AMZN'],
    'Rendite': [0.15, 0.12, 0.25, 0.18, 0.10],
    'Volatilitaet': [0.20, 0.18, 0.35, 0.22, 0.16],
    'Sharpe_Ratio': [0.75, 0.67, 0.71, 0.82, 0.63]
}

df = pd.DataFrame(data, index=['2023-01', '2023-02', '2023-03', '2023-04', '2023-05'])
df

,Aktie,Rendite,Volatilitaet,Sharpe_Ratio
2023-01,AAPL,0.15,0.20,0.75
2023-02,GOOG,0.12,0.18,0.67
2023-03,TSLA,0.25,0.35,0.71
2023-04,MSFT,0.18,0.22,0.82
2023-05,AMZN,0.10,0.16,0.63


### sort_values(): Sortieren nach Spaltenwerten

`sort_values()` sortiert den DataFrame nach einer oder mehreren Spalten. 
#### Einfaches sortieren nach einer Spalte
Sortieren wir nach der Sharpe-Ratio (absteigend, da höher besser ist).

In [10]:
# Sortiere nach Sharpe_Ratio absteigend
df_sorted_sharpe = df.sort_values(by='Sharpe_Ratio', ascending=False)
df_sorted_sharpe

,Aktie,Rendite,Volatilitaet,Sharpe_Ratio
2023-04,MSFT,0.18,0.22,0.82
2023-01,AAPL,0.15,0.20,0.75
2023-03,TSLA,0.25,0.35,0.71
2023-02,GOOG,0.12,0.18,0.67
2023-05,AMZN,0.10,0.16,0.63


#### Mehrspaltiges sortieren

Sortiere zuerst nach Rendite (absteigend), dann nach Volatilität (aufsteigend).

In [11]:
# Sortiere nach Rendite (abst.) und dann Volatilität (aufst.)
df_multi_sort = df.sort_values(by=['Rendite', 'Volatilitaet'], ascending=[False, True])
df_multi_sort

,Aktie,Rendite,Volatilitaet,Sharpe_Ratio
2023-03,TSLA,0.25,0.35,0.71
2023-04,MSFT,0.18,0.22,0.82
2023-01,AAPL,0.15,0.20,0.75
2023-02,GOOG,0.12,0.18,0.67
2023-05,AMZN,0.10,0.16,0.63


### sort_index(): Sortieren nach Index

#### Einfaches Index-Sortieren
Der Index ist bereits sortiert, aber ich simuliere einen ungesorteten Index:

In [12]:
# Unsortierter Index erstellen
df_unsorted = df.reindex(['2023-03', '2023-01', '2023-05', '2023-02', '2023-04'])
df_unsorted

,Aktie,Rendite,Volatilitaet,Sharpe_Ratio
2023-03,TSLA,0.25,0.35,0.71
2023-01,AAPL,0.15,0.20,0.75
2023-05,AMZN,0.10,0.16,0.63
2023-02,GOOG,0.12,0.18,0.67
2023-04,MSFT,0.18,0.22,0.82


Nun kann ich nach Index (hier: Daum) sortieren:

In [13]:
# Nun sortieren nach Index
df_sorted_index = df_unsorted.sort_index()
df_sorted_index

,Aktie,Rendite,Volatilitaet,Sharpe_Ratio
2023-01,AAPL,0.15,0.20,0.75
2023-02,GOOG,0.12,0.18,0.67
2023-03,TSLA,0.25,0.35,0.71
2023-04,MSFT,0.18,0.22,0.82
2023-05,AMZN,0.10,0.16,0.63


## Gruppieren & Aggregation

Die `groupby`-Funktion ist eines der mächtigsten Werkzeuge in Pandas. Sie ermöglicht es, Daten nach einer oder mehreren Spalten zu gruppieren und dann Aggregationen (Zusammenfassungen) über diese Gruppen durchzuführen. 
Anbei erstelle ich ein Beispieldokument, um die Anwendung besser darzustellen:

In [14]:
import pandas as pd
import numpy as np

data = {
    'Name': ['Anna', 'Ben', 'Clara', 'David', 'Emma'],
    'Alter': [28, 34, 19, 42, 31],
    'Gehalt_EUR': ['52.000,50', '68.000,00', '45.000,75', '82.000,00', '59.000,25'],
    'Eintritt': ['2021-03-15', '2020-11-01', '2023-07-20', '2019-01-10', '2022-05-05'],
    'Abteilung': ['IT', 'Vertrieb', 'IT', 'Vertrieb', 'Marketing']
}
df = pd.DataFrame(data)

df.to_csv('mitarbeiter_demo.csv', index=False, sep=';', decimal=',')
df = pd.read_csv(
    'mitarbeiter_demo.csv',
    sep=';', 
    encoding='utf-8', 
    decimal=',', 
    thousands='.', 
    parse_dates=['Eintritt'], 
    dtype={'Name': 'string', 'Abteilung': 'category'}
)

print(df.head())

    Name  Alter  Gehalt_EUR   Eintritt  Abteilung
0   Anna     28    52000.50 2021-03-15         IT
1    Ben     34    68000.00 2020-11-01   Vertrieb
2  Clara     19    45000.75 2023-07-20         IT
3  David     42    82000.00 2019-01-10   Vertrieb
4   Emma     31    59000.25 2022-05-05  Marketing


### Einfache Aggregationen (mean, sum, count, etc.)

Mit `groupby()` kann man Daten nach einer Kategorie gruppieren und dann eine einfache Aggregationsfunktion anwenden, wie `mean()` für den Durchschnitt, `sum()` für die Summe oder `count()` für die Anzahl. Die Funktion wird auf alle numerischen Spalten angewendet, es sei denn, man spezifiziert eine Spalte. Die `groupby('Spaltenname')` Funktion teilt den DataFrame in Gruppen nach der Spalte 'Spaltenname' auf. Dann wendet `mean()` den Durchschnitt auf die angegebene Spalte an. Der Parameter `observed=True` vermeidet Warnings bei kategorischen Spalten.


In [15]:
# Durchschnittsgehalt pro Abteilung
durchschnitt_gehalt = df.groupby('Abteilung', observed=True)['Gehalt_EUR'].mean()
print("Durchschnittsgehalt pro Abteilung:")
print(durchschnitt_gehalt)

# Summe der Alter pro Abteilung
summe_alter = df.groupby('Abteilung', observed=True)['Alter'].sum()
print("\nSumme der Alter pro Abteilung:")
print(summe_alter)

# Anzahl Mitarbeiter pro Abteilung
anzahl = df.groupby('Abteilung', observed=True).size()
print("\nAnzahl Mitarbeiter pro Abteilung:")
print(anzahl)

Durchschnittsgehalt pro Abteilung:
Abteilung
IT           48500.625
Marketing    59000.250
Vertrieb     75000.000
Name: Gehalt_EUR, dtype: float64

Summe der Alter pro Abteilung:
Abteilung
IT           47
Marketing    31
Vertrieb     76
Name: Alter, dtype: int64

Anzahl Mitarbeiter pro Abteilung:
Abteilung
IT           2
Marketing    1
Vertrieb     2
dtype: int64


### Mehrere Aggregationsfunktionen gleichzeitig (agg())
Mit `agg()` kann man mehrere Funktionen auf dieselbe Spalte oder verschiedene Funktionen auf unterschiedliche Spalten anwenden. Das Ergebnis ist ein DataFrame mit MultiIndexspalten.

Das Dictionary in `agg()` spezifiziert pro Spalte eine Liste von Funktionen (z. B. `['mean', 'sum']`). Pandas wendet diese auf jede Gruppe an und erstellt eine hierarchische Spaltenstruktur. 

In [16]:
# Ich führe zum vorherigen DataFrame eine weitere Spalte hinzu
df['Bonus_EUR'] = [5000, 8000, 3000, 10000, 6000]  # Beispiel: Bonus pro Mitarbeiter

# Mehrere Funktionen auf Gehalt und Bonus pro Abteilung angewendet
aggregation = df.groupby('Abteilung', observed=True).agg({
    'Gehalt_EUR': ['mean', 'sum', 'min'],  # Durchschnitt, Summe und Minimum des Gehalts
    'Bonus_EUR': ['max', 'std'],            # Maximum und Standardabweichung des Bonus
    'Alter': 'count'                        # Anzahl der Mitarbeiter (als 'count')
})
print("Mehrere Aggregationen pro Abteilung:")
print(aggregation)

print("\nNach Umbenennung der Spalten:")
print(aggregation)

Mehrere Aggregationen pro Abteilung:
          Gehalt_EUR                      Bonus_EUR              Alter
                mean        sum       min       max          std count
Abteilung                                                             
IT         48500.625   97001.25  45000.75      5000  1414.213562     2
Marketing  59000.250   59000.25  59000.25      6000          NaN     1
Vertrieb   75000.000  150000.00  68000.00     10000  1414.213562     2

Nach Umbenennung der Spalten:
          Gehalt_EUR                      Bonus_EUR              Alter
                mean        sum       min       max          std count
Abteilung                                                             
IT         48500.625   97001.25  45000.75      5000  1414.213562     2
Marketing  59000.250   59000.25  59000.25      6000          NaN     1
Vertrieb   75000.000  150000.00  68000.00     10000  1414.213562     2


Man kann auch alle Spalten auf einmal aggregieren:
Man kann `.agg()` auch ohne Dictionary verwenden, um dieselben Funktionen auf alle numerischen Spalten anzuwenden.
Bei `.agg(['mean', 'sum'])` werden alle Funktionen auf alle numerischen Spalten angewendet.

## Merges 
- concat() – Zeilen und Spalten zusammenhängen
- merge() – inner, left, right, outer
- join() auf Index
- Praxisbeispiel: Kundendaten + Bestelldaten verbinden

## Pivot-Tabellen

Pivot-Tabellen sind ein zentrales Werkzeug zur Datenaggregation und -umstrukturierung in `pandas`. Sie ermöglichen es, tabellarische Daten kompakt zusammenzufassen, etwa zur Auswertung von Zeitreihen, Verkaufszahlen oder versicherungsmathematischen Beständen.

### pivot() und pivot_table()

Die Methode `pivot()` dient zur reinen Umformung von Daten.  
Dabei darf jede Kombination aus Index und Spalten nur genau einen Wert besitzen. 





### margins & margins_name


In [17]:
import pandas as pd

df = pd.DataFrame({
    "Jahr": [2023, 2023, 2024, 2024],
    "Produkt": ["A", "B", "A", "B"],
    "Umsatz": [100, 150, 120, 180]
})

pivot_df = df.pivot(index="Jahr", columns="Produkt", values="Umsatz")
print(pivot_df)


Produkt    A    B
Jahr             
2023     100  150
2024     120  180


`pivot_table()` ist die verallgemeinerte Variante von `pivot()` und erlaubt:

- Aggregation (z. B. mean, sum, count)

- Mehrere Werte pro Kombination

- Fehlende Daten

In [18]:
df = pd.DataFrame({
    "Jahr": [2023, 2023, 2023, 2024, 2024],
    "Produkt": ["A", "A", "B", "A", "B"],
    "Umsatz": [100, 130, 150, 120, 180]
})

pivot_table_df = pd.pivot_table(
    df,
    values="Umsatz",
    index="Jahr",
    columns="Produkt",
    aggfunc="mean"
)

print(pivot_table_df)


Produkt      A      B
Jahr                 
2023     115.0  150.0
2024     120.0  180.0
